我们不能在机器学习中直接使用文本数据，所以我们需要将其转换为数值向量。本文章将为你介绍所有的文本转换技术。

 

> 在将文本数据提供给机器学习模型之前，需要将文本数据清洗并编码为数值，这种清洗和编码的过程称为**“文本预处理”**

 

在本篇文章中，我们将看到一些基本的文本清洗步骤和文本数据编码技术。 我们将会看到：

1. **了解数据** - 明白数据的全部含义。清理数据时应该考虑哪些事项（标点符号、停用词等）。

2. **基本清理** - 我们将看到清理数据时需要考虑哪些参数（例如标点符号，停用词等）及其代码。

3. 编码技术

    \- All the popular techniques that are used for encoding that I personally came across.

   - **Bag of Words**（词袋模型）
   - **Binary Bag of Words**（二进制词袋模型）
   - **Bigram, Ngram**
   - **TF-IDF**( **T**erm  **F**requency - **I**nverse **D**ocument **F**requency词频-逆文档频率)
   - **Word2Vec**
   - **Avg-Word2Vec**
   - **TF-IDF Word2Vec**

现在让我们开始这个有趣的旅程吧！
 

**导入库**

 

In [1]:
import warnings
warnings.filterwarnings("ignore")         # 忽略不必要的警告

import numpy as np                        # 用于大型和多维数组
import pandas as pd                       # 用于数据处理和分析
import nltk                               # 自然语言处理工具包

from nltk.corpus import stopwords         # 停用词语料库
from nltk.stem import PorterStemmer       # 词干提取器

from sklearn.feature_extraction.text import CountVectorizer    #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer    #For TF-IDF
from gensim.models import Word2Vec                             #For Word2Vec

In [2]:
data_path = "./input/Reviews.csv"
data = pd.read_csv(data_path)
data_sel = data.head(10000)                 # 只考虑前 10000 行

In [24]:
# 查看数据的结构
print(data_sel.columns)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')




1. **了解数据**

从数据集来看，我们的主要目标是根据文本预测评论是**正面**还是**负面**。

我们看到Score列，它的的取值有1,2,3,4,5。其中1、2为负面评价，4、5为正面评价。对于得分=3，我们将其视为中性评价，让我们删除中性行，以便可以预测正或负。

HelfulnessNumerator表示有多少人发现评论有用，HelpfulnessDenominator是关于usefull 评论数 + not so usefull 评论数的。由此我们可以看出，HelfulnessNumerator 总是小于或等于 HelpfulnessDenominator。

 

In [4]:
data_score_removed = data_sel[data_sel['Score']!=3]       # 去除中性评价


将分数值转换为类别标签Postiuve或Negative。

In [6]:
def partition(x):
    if x < 3:
        return 'positive'
    return 'negative'

score_upd = data_score_removed['Score']
t = score_upd.map(partition)  # Series.map方法，接受一个函数或含有映射关系的字典型对象
data_score_removed['Score']=t



1. **基本清洗**

**重复数据删除**表示删除重复的行，必须删除重复的行才能获得稳定的结果。 根据UserId，ProfileName，Time，Text检查重复项。 如果所有这些值都相等，那么我们将删除这些记录。 （没有用户可以在相同的确切时间输入不同产品的评论。）

我们已经知道HelpfulnessNumerator应该始终小于或等于HelpfulnessDenominator，因此检查此条件并删除这些记录。



In [7]:
final_data = data_score_removed.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})

In [8]:
final = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]

In [26]:
final_X = final['Text']
final_y = final['Score']



将所有单词转换为小写并删除标点符号和html标签（如果有）。

**词干提取**-将单词转换为基本词或词干（例如 - tastefully, tasty，这些单词将转换为词干“ tasti”）。 因为我们不考虑所有相似的词，所以这减小了向量维数。

**停用词** - 停用词是不必要的词，即使它们被删除了，句子的语义也不会发生变化。

例如 - This pasta is so tasty ==> pasta tasty    ( This , is, so 都是停用词，因此将它们删除)

参加以下代码单元，可以查看所有停用词。

 

In [10]:
stop = set(stopwords.words('english')) 
print(stop)

{'re', 'which', 'against', 'don', "won't", 'ours', "doesn't", 'other', 'very', 'weren', 'down', 'both', 'while', 'own', 'yourself', 'these', 'but', 'at', 'haven', 'between', 'doesn', 'y', 'it', "mightn't", 'been', 'won', 'its', 'mightn', 'ourselves', 'when', 'how', "wouldn't", 'who', 'once', 'their', 'her', 'each', 'then', 'for', 'you', "mustn't", 'so', 'am', "it's", 'here', 'wouldn', 'most', 'by', 'this', "that'll", 'what', 'being', 'off', 've', 'before', "aren't", "she's", 'shouldn', 'are', 'or', 'during', 'same', 'they', 'that', 'll', 'such', 'hadn', 'hasn', 'she', "couldn't", 'any', 'i', 'from', 'is', 'hers', 'after', "hasn't", 'were', 'over', "wasn't", 'more', "you've", 'he', 'with', 'wasn', 'nor', 'd', 'further', 'shan', 'out', 'them', 'on', "shouldn't", 'now', 'whom', 'as', "you'd", "you'll", 'because', 'all', 'too', 'my', 'no', "didn't", 'if', 'into', "weren't", 'does', 'than', 'mustn', 'himself', 'why', 'those', 'only', 'theirs', 'yours', 'couldn', 'up', 'me', "needn't", 'has'

In [27]:
import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in final_X:
    sentence = sentence.lower()                 # 转换为小写
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #将 HTML 标签替换为空格
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #将标点符号替换为空格
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # 删除停用词并提取词干
    temp.append(words)
    
final_X = temp   

In [28]:
print(final_X[1])

['product', 'arriv', 'label', 'jumbo', 'salt', 'peanut', 'peanut', 'actual', 'small', 'size', 'unsalt', 'sure', 'error', 'vendor', 'intend', 'repres', 'product', 'jumbo']


In [29]:
sent = []
for row in final_X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

final_X = sent
print(final_X[1])

 product arriv label jumbo salt peanut peanut actual small size unsalt sure error vendor intend repres product jumbo




1. **编码技术**

   **词袋模型**

   在BoW中，我们构建了一个词典，其中包含来自文本评论数据集中的所有不重复单词的集合。在这里会计算每一个单词出现的频率。 如果字典中有 **d** 个不重复词，则对于每个句子或评论，其向量的长度将为 **d**，并将来自评论的词数存储在向量中的特定位置。 在这种情况下，向量将会非常稀疏。

   例如. pasta is tasty and pasta is good

   **[0]....[1]............[1]...........[2]..........[2]............[1]..........**            <== 它的向量表示（其余所有点将表示为零）

   **[a]..[and].....[good].......[is].......[pasta]....[tasty].......**            <== 这是词典

   使用scikit-learn的CountVectorizer，我们可以获取BoW并检查其包含的所有参数，其中之一是max_features = 5000，它表示仅考虑将前5000个最频繁重复的单词放置在字典中。 因此我们的字典长度或向量长度只有5000。

**二进制词袋模型**

在二进制BoW中，我们不计算单词出现的频率，如果单词出现在评论中，我们就只将对应位置设置为 **1**，否则设置为 **0**。 在CountVectorizer中，有一个参数 **binary = true**，它可以使我们的BoW变为二进制BoW。

 

In [33]:
count_vect = CountVectorizer(max_features=5000)
bow_data = count_vect.fit_transform(final_X)  # 稀疏矩阵
print(bow_data[1])

  (0, 3641)	1
  (0, 2326)	1
  (0, 4734)	1
  (0, 1539)	1
  (0, 4314)	1
  (0, 4676)	1
  (0, 3980)	1
  (0, 4013)	1
  (0, 162)	1
  (0, 3219)	2
  (0, 3770)	1
  (0, 2420)	2
  (0, 2493)	1
  (0, 332)	1
  (0, 3432)	2


**词袋/二进制词袋模型的缺点**

我们在进行这些文本到矢量编码时的主要目标是，意思相近的文本矢量应该彼此接近，但是在某些情况下，这对于Bow来说是不可能的

例如，如果我们考虑两个评论 “ **This pasta is very tasty**” 和 “**This pasta is not tasty** ”，则在停用词移除后，这两个句子都将转换为 “ **pasta tasty**”，因此两者的含义完全相同。

主要问题是这里我们没有考虑与每个单词相关的前后词，于是就有了Bigram和Ngram技术。

 

**BI-GRAM 词袋模型**

考虑到用于创建字典的单词对为Bi-Gram，因此Tri-Gram表示三个连续的单词，以此类推到NGram。

CountVectorizer 有一个参数 **ngram range** 如果分配给（1,2），表示为 Bi-Gram BoW。

但这极大地增加了我们的字典容量。

In [35]:
final_B_X = final_X

In [39]:
count_vect = CountVectorizer(ngram_range=(1,2))
Bigram_data = count_vect.fit_transform(final_B_X)
print('Shape:', Bigram_data[1].shape)
print(Bigram_data[1])

Shape: (1, 207785)
  (0, 143171)	1
  (0, 151696)	1
  (0, 95087)	1
  (0, 196648)	1
  (0, 60866)	1
  (0, 177168)	1
  (0, 193567)	1
  (0, 164722)	1
  (0, 165627)	1
  (0, 4021)	1
  (0, 133855)	1
  (0, 133898)	1
  (0, 155987)	1
  (0, 97865)	1
  (0, 100490)	1
  (0, 11861)	1
  (0, 142800)	1
  (0, 151689)	1
  (0, 95076)	1
  (0, 196632)	1
  (0, 60852)	1
  (0, 177092)	1
  (0, 193558)	1
  (0, 164485)	1
  (0, 165423)	1
  (0, 3831)	1
  (0, 133854)	2
  (0, 155850)	1
  (0, 97859)	2
  (0, 100430)	1
  (0, 11784)	1
  (0, 142748)	2


 

**词频 - 逆文档频率（TF-IDF）**

**词频 - 逆文档频率**，它可以确保对最频繁使用的单词给予较少的重视，同时也考虑低频使用的单词。

**词频**是在评论中出现 **特定单词（W）**的次数除以评论中的总单词数量 **（Wr）**。 词频值的范围是0到1。

**逆文档频率 **的计算为 **log（文档总数（N）/包含特定单词的文档数（n））**。 这里的文档称为 “评论”。

**词频 - 逆文档频率(TF-IDF)** 等于 $TF*IDF$ 即 $\dfrac{W*log{\frac Nn}}{Wr}$ 

使用scikit-learn的tfidfVectorizer，我们可以获得TF-IDF。

那么，即使在这里，我们为每个单词都获得了TF-IDF值，在某些情况下，删除停用词后，它还是可能会将不同含义的评论视为是相似的。 因此，我们可以使用BI-Gram或NGram模型。

 

In [43]:
final_tf = final_X
tf_idf = TfidfVectorizer(max_features=5000)
tf_data = tf_idf.fit_transform(final_tf)
print('Shape:', tf_data.shape)
print(tf_data[1])

Shape: (8718, 5000)
  (0, 3432)	0.1822092004981035
  (0, 332)	0.1574317775964303
  (0, 2493)	0.18769649750089953
  (0, 2420)	0.5671119742041831
  (0, 3770)	0.1536626385509959
  (0, 3219)	0.3726548417697838
  (0, 162)	0.14731616688674187
  (0, 4013)	0.14731616688674187
  (0, 3980)	0.14758995053747803
  (0, 4676)	0.2703170210936338
  (0, 4314)	0.14376924933112933
  (0, 1539)	0.2676489579732629
  (0, 4734)	0.22110622670603633
  (0, 2326)	0.25860104128863787
  (0, 3641)	0.27633136515735446


 

因此，要真正克服距离较近的语义评论的问题，我们需要使用Word2Vec。

**Word2Vec**

Word2Vec实际上采用了单词的语义含义以及它们与其他单词之间的关系。 它学习单词之间的所有内部关系，以密集的向量形式表示单词。

使用 **Gensim** 的库，我们就可以调用 Word2Vec，它接收的是像 **min_count = 5** 之类的参数，表示仅当单词在整个数据中重复超过5次时才会考虑。 **size = 50** 表示矢量长度为50，而 **workers** 表示的是运行此代码的核数。

**Average Word2Vec**

计算每个单词的Word2vec，将每个单词的向量相加，然后将向量除以句子的单词数，简单地求出每条评论中所有单词的Word2vec的平均值。

 

In [45]:
w2v_data = final_X

In [46]:
splitted = []
for row in w2v_data: 
    splitted.append([word for word in row.split()])     #分词

In [53]:
train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)

In [54]:
avg_data = []
for row in splitted:
    vec = np.zeros(50)
    count = 0
    for word in row:
        try:
            vec += train_w2v[word]
            count += 1
        except:
            pass
    avg_data.append(vec/count)
    

In [55]:
print(avg_data[1])

[-0.45812476  0.08458711  0.05092981  0.14410483 -0.49741123 -0.20359915
  0.09425076  0.31108052 -0.39218421  0.13923831 -0.03049097 -0.11326336
 -0.15359964  0.15278533 -0.12085474 -0.09121271  0.10288595 -0.28526957
 -0.22663371 -0.41127726  0.00271141  0.42055635 -0.23583491 -0.12041335
 -0.25532713 -0.12806206 -0.16156811 -0.57879642 -0.29231823  0.29289352
  0.07047758 -0.05602346  0.15903048  0.00325343  0.01416281 -0.26845292
  0.24614703 -0.32351884 -0.37319936 -0.05872503 -0.2490074  -0.3033209
 -0.75307091  0.19496457 -0.22516627  0.02649581  0.14338367  0.67597507
 -0.64155409  0.07120903]


 

**TF-IDF WORD2VEC**

在TF-IDF Word2Vec中，每个单词的Word2Vec值乘以该单词的tfidf值，然后求和，然后除以句子的tfidf值之和。

就像是：

```python
                    V = ( t(W1)*w2v(W1) + t(W2)*w2v(W2) +.....+t(Wn)*w2v(Wn))/(t(W1)+t(W2)+....+t(Wn))
```

 

In [108]:
tf_w_data = final_X
tf_idf = TfidfVectorizer()
# tf_idf = TfidfVectorizer(max_features=5000)
tf_idf_data = tf_idf.fit_transform(tf_w_data)
# print(tf_idf.vocabulary_)

In [109]:
tf_w_data = []
tf_idf_data = tf_idf_data.toarray()
i = 0
for row in splitted:
    vec = [0 for i in range(50)]
    
    temp_tfidf = []
    for val in tf_idf_data[i]:
        if val != 0:
            temp_tfidf.append(val)
    
    count = 0
    tf_idf_sum = 0
    for word in row:  # 注释: 此处如何确保tf-idf的单词顺序与评论中的单词顺序一一对应???
        try:
            count += 1
            tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
            vec += (temp_tfidf[count-1] * train_w2v[word])  # 不能保证是同一个单词的tfidf值和Word2Vec值
        except:
            pass
    vec = (float)(1/tf_idf_sum) * vec
    tf_w_data.append(vec)
    i = i + 1

print(tf_w_data[1])

[-3.65244400e-01  4.57776099e-02  9.98857835e-02  1.97531826e-01
 -4.82579953e-01 -1.04751093e-01  4.78762299e-02  2.98722871e-01
 -4.82290494e-01  9.07129827e-02 -2.50973641e-01 -2.97829000e-01
 -5.46013241e-02  1.49918537e-01 -3.34651424e-01 -1.32334573e-01
  4.21436157e-02 -3.42838623e-01 -6.32972503e-03 -2.94089193e-01
  1.89783762e-01  6.10301421e-01 -3.90488853e-01 -1.11587678e-01
 -1.86647763e-01 -2.65755767e-01 -6.99710703e-02 -5.71435860e-01
 -1.34036573e-01  5.06373095e-01  1.20161662e-01 -2.63341359e-01
  2.89655554e-02  6.42477261e-02  2.50360920e-01 -2.98312006e-01
  3.39811540e-01 -4.62875199e-01 -3.81373302e-01 -2.46285669e-01
 -4.59487886e-01 -4.13959972e-01 -9.44517293e-01  3.63023605e-01
 -4.07729702e-01 -2.93615766e-02  1.58460594e-01  8.06446577e-01
 -7.49817592e-01  1.73608258e-04]


 

**结论**

通过本文我们看到了将文本数据编码为数值向量的不同技术。 但是哪种技术适合我们的机器学习模型还要取决于数据的结构和模型的目标。